<a href="https://colab.research.google.com/github/1830668NabilahOshin/Numerical-methods-and-Neural-Network-Labwork/blob/main/appending_short_october.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from googleapiclient.discovery import build
!pip install gspread
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=7ad350259da8a970be822b16f18c014127326fe84104ea02066ba106cbffe026
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect


In [ ]:
API_KEY = "*****************"

def extract_comments(video_id):
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    video_response = youtube.videos().list(part='snippet', id=video_id).execute()
    video_title = video_response['items'][0]['snippet']['title']

    comments = []
    next_page_token = None
    existing_comments = set()

    while True:
        comments_response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            pageToken=next_page_token,
            maxResults=200,
            order='relevance'
        ).execute()

        for comment in comments_response['items']:
            comment_text = comment['snippet']['topLevelComment']['snippet']['textDisplay']
            if is_bengali(comment_text) and is_long_comment(comment_text, 4) and comment_text not in existing_comments:
                comments.append(comment_text)
                existing_comments.add(comment_text)
                if len(comments) == 60:
                    return video_title, comments

        next_page_token = comments_response.get('nextPageToken')

        if not next_page_token:
            break

    return video_title, comments

#def is_bengali(text):
    #bengali_pattern = re.compile(r'^[\u0980-\u09FF\s]+$')
    #return bool(bengali_pattern.match(text))

def is_bengali(text):
    # Update the regular expression pattern to include a wider range of characters
    bengali_pattern = re.compile(r'^[a-zA-Z0-9\u0980-\u09FF\s.,!?😀-😮]+$')
    return bool(bengali_pattern.match(text))

def is_long_comment(text, min_words):
    words = text.split()
    return len(words) > min_words


video_ids2 = ["MfdIW-gFk4w", "Fl1RKHdU-IA","emC-PO20t-k","QoXfL33QWbc","JIdFx3uJC6E","-tLHhA-TH6I","gXswrTUWZHc"]
video_ids3 = []
#commas = ["", "",]



In [ ]:
def is_bengali(text):
    try:
        language = detect(text)
        if language == "bn":
            # Check if the comment contains emojis, punctuations, or special characters
            if any(char in text for char in ".,!?😀-😮"):
                return True
        return False
    except:
        return False

In [ ]:
# Authenticate and open the Google Sheets file
credentials_path = "/content/drive/MyDrive/moo-high-men/banglaytcomments-73184086a64a.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
client = gspread.authorize(credentials)

# Specify the Google Sheets file name or URL
gsheet_file = "Bangla Error Corrector Dataset from YouTube Comments"

# Open the Google Sheets file
gsheet = client.open(gsheet_file)

# Select the first sheet in the Google Sheets file
worksheet = gsheet.get_worksheet(8)

# Get all existing comments from the worksheet
existing_comments = set(worksheet.col_values(2)[1:])



In [ ]:
import pandas as pd

# ... (previous code)

# Create an empty DataFrame to store video title and comments
comments_df = pd.DataFrame(columns=['Video Title', 'Comment'])

rows_data = []
for video_id in video_ids3:
    video_title, comments = extract_comments(video_id)
    for comment in comments:
        if comment not in existing_comments:
            comments_df = comments_df.append({'Video Title': video_title, 'Comment': comment}, ignore_index=True)
            rows_data.append([video_title, comment])
            existing_comments.add(comment)



worksheet.append_rows(rows_data)

# Get the URL of the Google Sheets file
gsheet_url = gsheet.url

# Print the URL
print(f"Google Sheets file URL: {gsheet_url}")

# Print the path to the Google Sheets file
print(f"Bengali comments appended to {gsheet_file}")






In [ ]:
print(comments_df.shape)

(60, 2)


# =========================================================================

In [ ]:
print(comments_df.shape)

(60, 5)
